# 太陽と地球と月
太陽と公転する地球、月を描画するコードです。
それっぽく見えるだけで実態は円の公式に沿った描画を行います。
動作確認済み環境は以下の通り。

項目|バージョン
---|---
OS|Mac OSX 10.10.5(Yosemite)
Python|3.5.1
matplotlib|1.5.1
numpy|1.11.0
ipython|4.1.2

## 必要なライブラリのインポート

In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML,display
import math


## プロット用のオブジェクトの作成
動画は基本的にパラパラ漫画なので、動かすグラフは前もってインスタンスを生成し、  
更新用のメソッド内に渡してあげる必要があります。

In [10]:
# プロット用のオブジェクトを作成
fig, ax = plt.subplots()
plt.close()
# 描画領域の縦、横のサイズを指定
fig.set_figheight(6.0)
fig.set_figwidth(6.0)
# X軸を-1.5〜1.5までに設定
ax.set_xlim((-1.5, 1.5))
# Y軸を-1.5〜1.5までに設定
ax.set_ylim((-1.5, 1.5))

# 地球描画用のオブジェクトを作成
earth = ax.scatter([],[], color='blue',marker='o',s=80)
# 月描画用のオブジェクトを作成
moon = ax.scatter([],[], color='gray',marker='o',s=40)



## 太陽の描画
太陽自体は動画の中では動かすものではないので、  
ここで先に描画して、背景のように扱います。

In [11]:
ax.scatter(0,0,color='red',marker='o',s=320)

## 更新用メソッドの定義
円の方程式について中心が原点で半径がrの場合は以下の通りとなります。
\begin{eqnarray}
x^2+y^2&=&r^2\\
x&=&r\cos \theta\\
y&=&r\sin \theta\\
\end{eqnarray}  
上記で地球の公転がか描画できます。更に、中心が原点から離れている場合は以下の通りとなります。
\begin{eqnarray}
(x-a)^2+(y-b)^2&=&r^2\\
x&=&a+r\cos \theta\\
y&=&b+r\sin \theta\\
\end{eqnarray}  
$a$と$b$を以下のように移動するような値$\cos$や$\sin$にすることで月が地球を公転しながら太陽の周りも回るような描画が可能となります。  
\begin{eqnarray}
(x-a)^2+(y-b)^2&=&r^2\\
x&=&\cos \theta+r\cos \theta\\
y&=&\sin \theta+r\sin \theta\\
\end{eqnarray}  
また$\theta$の値を増加させることで公転速度が速くなるような描画可能となります。

In [12]:
# コールバックメソッドの定義
def animate(i,earth,moon):
    # フレームからラジアンを算出
    # ラジアンの算出式は「度数 × 円周率 ÷ 180」
    x =  i * math.pi / 180
    # フレームに対してxの移動を2倍速にする
    x = 2 * x 
    
    # 地球の描画
    earth_x = math.cos(x) 
    earth_y = math.sin(x) 
    earth.set_offsets((earth_x, earth_y))

    # 月の描画
    moon_x = math.cos(x) + 1/8 * math.cos(4 * x)
    moon_y = math.sin(x) + 1/8 * math.sin(4 * x)
    moon.set_offsets((moon_x,moon_y))

## アニメーションの生成
ここでFuncAnimationメソッドを利用してアニメーションを生成します。
メソッドの引数は以下の通りです。

引数|意味
---|---
fig|描画領域
func|フレーム毎に更新する描写を定義したコールバック関数
fargs|更新するグラフの関数
frames|描画全体のフレーム数
interval|何ミリ秒毎に再描画するか（アニメーションのコマを進めるか）

In [13]:
# 動画全体のフレームを設定する
FRAMES = 180

anim = animation.FuncAnimation(fig = fig, func = animate, fargs = (earth,moon), frames = FRAMES, interval=30)

## アニメーションの出力
アニメーションの出力先によって方法が異なります。以下の通り

出力先|方法
---|---
IPython内に表示する|HTMLクラスを利用する
MP4やGIF等ファイルに出力する|Writerクラスを利用する

また、Writerクラスをインスタンス化する際の引数は以下の通りです。

引数|意味
---|---
fps|フレームレート（ほとんどの動画サイトは30fps以下であればアップロードできる。<br>また、そんなにないと思うけれどDVD化等に動画を出力する際は<br>29.97fpsにする必要がある。そのため、ここでは29.97に設定）
metadata|動画作成者等のメタデータを設定
bitrate|動画のビットレート（画質）

In [14]:
export_to = "IPython"
#export_to = "MP4"
#export_to = "GIF"
title = "sun_earth_moon"

if export_to == "IPython":
    # HTML化することで描画
    display(HTML(anim.to_html5_video()))
elif export_to == "MP4":
    # FFmpegというUNIX系の動画変換ソフトを利用する
    Writer = animation.writers['ffmpeg']
    # エクスポートされるファイルの情報を定義してインスタンス化する
    writer = Writer(fps=29.97, metadata=dict(artist='Me'), bitrate=1800)
    # 実際の動画のファイルエスクスポート
    anim.save(title + '.mp4', writer=writer)
elif export_to == "GIF":
    # FFmpegというUNIX系の動画変換ソフトを利用する
    Writer = animation.writers['imagemagick']
    # エクスポートされるファイルの情報を定義してインスタンス化する
    writer = Writer(fps=29.97, metadata=dict(artist='Me'), bitrate=1800)
    # 実際の動画のファイルエスクスポート
    anim.save(title + '.gif', writer=writer)
